In [ ]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
primary_df = pl.read_csv('primary_data.csv')

In [ ]:
secondary_df = pl.read_csv('secondary_data.csv')

In [ ]:
reference_df = pl.read_csv('primary_data_reference.csv')

In [ ]:
primary_df = primary_df.with_columns(
    (pl.col("associate_date").is_not_null()).alias("ACAS"),
    (pl.col("Fellow_Date").is_not_null()).alias("FCAS")
)
primary_df = primary_df.with_columns(
    pl.when(pl.col("FCAS"))
      .then(pl.lit("FCAS"))
      .when(pl.col("ACAS"))
      .then(pl.lit("ACAS"))
      .otherwise(pl.lit("non-member"))
      .alias("membership_type")
)

In [ ]:
pdf_pie_IP = primary_df.filter(pl.col('Delivery')=='IP').to_pandas()
pdf_pie_LS = primary_df.filter(pl.col('Delivery')=='LS').to_pandas()
pdf_pie_V = primary_df.filter(pl.col('Delivery')=='V').to_pandas()
pdf_pie_all = primary_df.to_pandas()

In [ ]:
import matplotlib.pyplot as plt

def plot_pies(df, group_col, event_col="Event_Code"):
    event_codes = df[event_col].unique()
    n = len(event_codes)

    fig, axes = plt.subplots(1, n, figsize=(4*n, 4))

    if n == 1:
        axes = [axes]

    for ax, event in zip(axes, event_codes):
        subset = df[df[event_col] == event]
        counts = subset[group_col].value_counts()

        ax.pie(counts, labels=counts.index, autopct="%1.1f%%")
        ax.set_title(f"{event} - {group_col}")

    plt.show()


IP

In [ ]:
plot_pies(pdf_pie_IP, "registrant_type_code")
plot_pies(pdf_pie_IP, "membership_type")
plot_pies(pdf_pie_IP, "State")
plot_pies(pdf_pie_IP, "Country")

LS

In [ ]:
plot_pies(pdf_pie_LS, "registrant_type_code")
plot_pies(pdf_pie_LS, "membership_type")
plot_pies(pdf_pie_LS, "State")
plot_pies(pdf_pie_LS, "Country")

V

In [ ]:
plot_pies(pdf_pie_V, "registrant_type_code")
plot_pies(pdf_pie_V, "membership_type")
plot_pies(pdf_pie_V, "State")
plot_pies(pdf_pie_V, "Country")

all events

In [ ]:
# plot_pies(pdf_pie_all, "registrant_type_code")
# plot_pies(pdf_pie_all, "membership_type")
# plot_pies(pdf_pie_all, "State")
# plot_pies(pdf_pie_all, "Country")

In [ ]:
primary_df_parent_events = [
    "2015RPM", "15Spring", "2015REINS", "15CLRS", "15CLRS", "15Annual", "2016RPM", "16Spring", "16Spring", "2016REINS",
    "16CLRS", "16CLRS", "16Annual", "17RPM", "17Spring", "17Spring", "2017REINS", "17CLRS", "17CLRS", "17Annual",
    "17Annual", "18RPM", "18Spring", "18Spring", "2018REINS", "18CLRS", "18CLRS", "18Annual", "18Annual", "19RPM",
    "19RPM", "19Spring", "19Spring", "2019REINS", "2019REINS", "19CLRS", "19CLRS", "19Annual", "19Annual", "20VirSprng",
    "20VREINS", "20VRPM", "20VRCLRS", "20VirAnn", "21VRPM", "21VirSprng", "21VREINS", "21VRCLRS", "21Annual", "21Annual",
    "RPMLive22", "22Spring", "22Spring", "22VREINS", "22CLRS", "22CLRS", "22Annual", "22Annual", "23RPM", "23RPM",
    "23Spring", "23Spring", "23REI", "23REI", "23CLRS", "23CLRS", "23Annual", "23Annual", "24RPM", "24RPM",
    "24Spring", "24Spring", "24Reins", "24Reins", "24CLRS", "24CLRS", "24Annual", "24Annual",
]


In [ ]:
def add_event_type_column(df: pl.DataFrame) -> pl.DataFrame:
    return df.with_columns(
    pl.when(pl.col("parent_event").str.to_lowercase().str.contains("ann"))
      .then(pl.lit("Annual"))
    .when(pl.col("parent_event").str.to_lowercase().str.contains("clrs"))
      .then(pl.lit("CLRS"))
    .when(pl.col("parent_event").str.to_lowercase().str.contains("rpm"))
      .then(pl.lit("RPM"))
    .when(pl.col("parent_event").str.to_lowercase().str.contains("rei"))
      .then(pl.lit("REINS"))
    .when(pl.col("parent_event").str.to_lowercase().str.contains("spr"))
      .then(pl.lit("Spring"))
    .otherwise(pl.lit("type not found"))
    .alias("event_type")
)

In [ ]:
primary_df_grouped = (
    primary_df.group_by("Event_Title")
      .agg([
          pl.col("Event_Code").first().alias("Event_Code"),

          pl.col("Start_Date").first().alias("Start_Date"),

          pl.col("Delivery").first().alias("Delivery"),

          pl.col("Location_City").first().alias("Location_City"),

          pl.col("Location_State").first().alias("Location_State"),

          pl.len().alias("attendance"),
      ])
).with_columns(
        pl.col("Start_Date")
        .str.strptime(pl.Date, "%m/%d/%Y", strict=False)
        .alias("Start_Date")
).sort(by='Start_Date').with_columns(
   pl.Series("parent_event", primary_df_parent_events)
    )

In [ ]:
primary_df_grouped = add_event_type_column(primary_df_grouped)
primary_df_grouped = primary_df_grouped.with_columns(pl.concat_str([pl.col("Location_City"), pl.lit(", "), pl.col("Location_State")]).alias("city_state"))

totals_ip_ls = (
    primary_df_grouped
    .filter(pl.col("Delivery").is_in(["IP", "LS"]))
    .group_by("parent_event")
    .agg([
        pl.col("attendance").sum().alias("attendance"),
        pl.lit("IP+LS").alias("Delivery"),

        pl.col("Location_City").first().alias("Location_City"),
        pl.col("Location_State").first().alias("Location_State"),
        pl.col("city_state").first().alias("city_state"),
        pl.col("event_type").first().alias("event_type"),
        pl.col("Start_Date").first().alias("Start_Date"),

        pl.lit(None, dtype=pl.Utf8).alias("Event_Title"),
        pl.lit(None, dtype=pl.Utf8).alias("Event_Code"),
    ])
)
primary_df_grouped_combined = pl.concat([primary_df_grouped, totals_ip_ls], how="diagonal")

In [ ]:
pdf_line_CLRS = primary_df_grouped_combined.filter(pl.col('event_type')=='CLRS').to_pandas()
pdf_line_Annual = primary_df_grouped_combined.filter(pl.col('event_type')=='Annual').to_pandas()
pdf_line_RPM = primary_df_grouped_combined.filter(pl.col('event_type')=='RPM').to_pandas()
pdf_line_REINS = primary_df_grouped_combined.filter(pl.col('event_type')=='REINS').to_pandas()
pdf_line_Spring = primary_df_grouped_combined.filter(pl.col('event_type')=='Spring').to_pandas()

In [ ]:
def plot_line_by_delivery(df, event_type):
    plt.figure(figsize=(12,6))
    ax = sns.lineplot(
        data=df,
        x="Start_Date",
        y="attendance",
        marker="o",
        hue="Delivery"
    )
    for (x, y, txt) in zip(df["Start_Date"], df["attendance"], df['city_state']):
        ax.annotate(
            txt,
            xy=(x, y),
            xytext=(0, 6),
            textcoords="offset points",
            ha="center",
            va="bottom",
            fontsize=8
        )
    
    plt.title(f"{event_type} Event Attendance Over Time", fontsize=16)
    plt.xlabel("Event Date")
    plt.ylabel("Attendance")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_line_by_delivery(pdf_line_CLRS, "CLRS")
plot_line_by_delivery(pdf_line_Annual, "Annual")
plot_line_by_delivery(pdf_line_RPM, "RPM")
plot_line_by_delivery(pdf_line_REINS, "REINS")
plot_line_by_delivery(pdf_line_Spring, "Spring")

In [ ]:
event_type_order = ["Annual", "CLRS", "REINS", "RPM", "Spring"]

colors = sns.color_palette("tab10", n_colors=len(event_type_order))
EVENT_TYPE_TO_COLOR = dict(zip(event_type_order, colors))

pdf_line_IP = primary_df_grouped_combined.filter(pl.col('Delivery')=='IP').to_pandas()
pdf_line_LS = primary_df_grouped_combined.filter(pl.col('Delivery')=='LS').to_pandas()
pdf_line_IPLS = primary_df_grouped_combined.filter(pl.col('Delivery')=='IP+LS').to_pandas()

for df in (pdf_line_IP, pdf_line_LS, pdf_line_IPLS):
    df["event_type"] = pd.Categorical(df["event_type"], categories=event_type_order)

In [ ]:
def plot_line_by_event_type(df, event_type):
    plt.figure(figsize=(12,6))
    ax = sns.lineplot(
        data=df,
        x="Start_Date",
        y="attendance",
        marker="o",
        hue="event_type",
        hue_order=event_type_order,
        palette=EVENT_TYPE_TO_COLOR
    )
    for (x, y, txt) in zip(df["Start_Date"], df["attendance"], df['city_state']):
        ax.annotate(
            txt,
            xy=(x, y),
            xytext=(0, 6),
            textcoords="offset points",
            ha="center",
            va="bottom",
            fontsize=8
        )
    
    plt.title(f"{event_type} Event Attendance Over Time", fontsize=16)
    plt.xlabel("Event Date")
    plt.ylabel("Attendance")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_line_by_event_type(pdf_line_IP, "IP")
plot_line_by_event_type(pdf_line_LS, "LS")
plot_line_by_event_type(pdf_line_IPLS, "IP+LS")

In [ ]:
secondary_event_types = [
    'CLRS', 'CLRS', 'ANN', 'ANN', 'RPM', 'RPM', 'SPR', 'SPR', 'SPR', 'REI', 'REI', 'CLRS',
    'CLRS', 'CLRS', 'CSAF', 'ANN', 'ANN', 'ANN', 'ANN', 'RPM', 'RPM', 'SPR', 'SPR', 'REI',
    'REI', 'CLRS', 'CLRS', 'ANN', 'ANN', 'ANN', 'RPM', 'RPM', 'SPR', 'SPR', 'REI', 'REI',
    'CLRS', 'CLRS', 'ANN', 'ANN', 'ANN', 'RPM', 'RPM', 'RPM', 'SPR', 'SPR', 'REI', 'REI',
    'CLRS', 'CLRS', 'ANN', 'ANN', 'RPM', 'RPM', 'RPM', 'RPM', 'SPR', 'SPR', 'SPR', 'REI',
    'REI', 'CARE', 'CLRS', 'CLRS', 'ANN', 'ANN', 'ANN', 'RPM', 'SPR', 'SPR', 'SPR', 'REI',
    'REI', 'REI', 'REI', 'RPM', 'CLRS', 'CLRS', 'CLRS', 'CLRS', 'CSAF', 'ANN', 'ANN', 'RPM',
    'RPM', 'RPM', 'RPM', 'SPR', 'SPR', 'SPR', 'REI', 'REI', 'REI', 'REI', 'CLRS', 'CLRS',
    'CLRS', 'CLRS', 'ANN', 'ANN', 'RPM', 'RPM', 'RPM', 'RPM', 'SPR', 'SPR', 'REI', 'REI',
    'CLRS', 'CLRS', 'CLRS', 'CLRS', 'ANN', 'RPM', 'RPM', 'RPM', 'SPR', 'REI', 'REI', 'REI',
    'CLRS', 'CLRS', 'CLRS', 'ANN', 'RPM', 'RPM', 'SPR', 'REI', 'REI', 'REI', 'CLRS', 'CLRS',
    'CLRS', 'ANN', 'RPM', 'RPM', 'RPM', 'SPR', 'REI', 'REI', 'REI'
]


In [ ]:
secondary_df_grouped = secondary_df.group_by('Meeting').agg([pl.col('Begin_Date').first(), pl.len().alias('attendance')]).with_columns(
        pl.col("Begin_Date")
        .str.strptime(pl.Date, "%m/%d/%Y", strict=False)
        .alias("Begin_Date")
).sort(by='Begin_Date').with_columns(
   pl.Series("event_type", secondary_event_types),
    pl.col("Meeting").str.slice(0, 4).alias("Year")
    ).with_columns(pl.concat_str(
        [pl.col("Year"), pl.col("event_type")],
        separator=""
    ).alias("year_and_event"))

secondary_event_totals = secondary_df_grouped.group_by('year_and_event').agg([pl.col('attendance').sum(), pl.col('Begin_Date').first(), pl.col('event_type').first()]).sort(by='Begin_Date')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

pdf = secondary_event_totals.filter(pl.col('year_and_event')!='2014ANN').to_pandas()  # Remove 2014 Annual as it was the CAS Centenniel Celebration and was an outlier, distorting the chart.

fig, ax = plt.subplots(figsize=(10, 5))
sns.lineplot(
    data=pdf,
    x="Begin_Date",
    y="attendance",
    hue="event_type",
    marker="o",
    ax=ax,
)

ax.set_title("Attendance over time by event type")
ax.set_xlabel("Date")
ax.set_ylabel("Attendance")
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
ax.grid(True, alpha=0.3)

ax.legend(title="Event type", frameon=False, bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()
plt.show()


Location Work

In [ ]:
import json
from pathlib import Path
from typing import Iterable

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

CACHE_PATH = Path("geocode_cache.json")

def load_cache() -> dict:
    if CACHE_PATH.exists():
        return json.loads(CACHE_PATH.read_text(encoding="utf-8"))
    return {}

def save_cache(cache: dict) -> None:
    tmp = CACHE_PATH.with_suffix(".tmp")
    tmp.write_text(json.dumps(cache, ensure_ascii=False, indent=2), encoding="utf-8")
    tmp.replace(CACHE_PATH)


def build_unique_place_keys(primary_df) -> list[str]:
    people_keys = primary_df.select(
        pl.concat_str(
            [pl.col("City"), pl.col("State"), pl.col("Country")],
            separator=", ",
            ignore_nulls=True,
        ).alias("place_key")
    )
    meeting_keys = primary_df.select(
        pl.concat_str(
            [pl.col("Location_City"), pl.col("Location_State")],
            separator=", ",
            ignore_nulls=True,
        ).alias("place_key")
    )
    out = (
        pl.concat([people_keys, meeting_keys])
        .unique()
        .filter(pl.col("place_key").str.len_chars() > 0)
        .get_column("place_key")
        .to_list()
    )
    return out

def geocode_all(unique_place_keys: Iterable[str], save_every: int = 50) -> None:
    cache = load_cache()

    geolocator = Nominatim(user_agent="meetings-distance")
    geocode = RateLimiter(
        geolocator.geocode,
        min_delay_seconds=1,         
        max_retries=2,               
        error_wait_seconds=5,
        swallow_exceptions=True,     
    )

    calls_since_save = 0
    retry = [k for k, v in cache.items() if (v.get('lat') is None or v.get('lon') is None)]
    todo = [k for k in unique_place_keys if k and k not in cache] + retry
    print(f"{len(todo)} places to geocode (skipping {len(unique_place_keys) - len(todo)} already cached).")

    for idx, key in enumerate(todo, start=1):
        loc = geocode(key)
        if loc:
            cache[key] = {
                "lat": loc.latitude,
                "lon": loc.longitude,
                "display_name": getattr(loc, "address", None),
            }
            print(f"[{idx}/{len(todo)}] OK  - {key} -> ({loc.latitude:.6f}, {loc.longitude:.6f})")
        else:
            cache[key] = {"lat": None, "lon": None, "display_name": None}
            print(f"[{idx}/{len(todo)}] MISS- {key}")

        calls_since_save += 1
        if calls_since_save >= save_every:
            save_cache(cache)
            print(f"Saved cache after {calls_since_save} new calls.")
            calls_since_save = 0

    save_cache(cache)
    print("All done. Cache saved to", str(CACHE_PATH))

In [ ]:
# unique_place_keys = build_unique_place_keys(primary_df)
# geocode_all(unique_place_keys, save_every=50)

In [ ]:
cache = json.loads(open("geocode_cache.json", encoding="utf-8").read())
cache_df = pl.DataFrame({
    "place_key": list(cache.keys()),
    "lat": [v["lat"] for v in cache.values()],
    "lon": [v["lon"] for v in cache.values()],
})

In [ ]:
# people_locations = primary_df.select(
#         pl.concat_str(
#             [pl.col("City"), pl.col("State"), pl.col("Country")],
#             separator=", ",
#             ignore_nulls=True,
#         ).alias("place_key"))

# n = 0
# for p in people_locations['place_key']:
#     if p in cache_df['place_key'] and not len(cache_df.filter(pl.col('place_key')==p).filter(pl.col('lat').is_null()))>0:
#         n +=1
# print(n/len(people_locations))

99% of participants' locations returned valid coordinates

In [ ]:
# meeting_locations = primary_df.select(
#         pl.concat_str(
#             [pl.col("Location_City"), pl.col("Location_State")],
#             separator=", ",
#             ignore_nulls=True,
#         ).alias("place_key")
#     ).filter(pl.col('place_key')!="")

# n2 = 0
# for m in meeting_locations['place_key']:
#     if m in cache_df['place_key'] and not len(cache_df.filter(pl.col('place_key')==m).filter(pl.col('lat').is_null()))>0:
#         n2 +=1
# print(n2/len(meeting_locations))

100% of meeting locations have coordinates (important)

In [ ]:
event_title = [
    '2015 CAS Annual Meeting', '2015 CAS Interactive Live Streaming: CLRS', '2015 CAS Spring Meeting',
    '2015 CLRS & Workshops', '2015 Ratemaking and Product Management Seminar & Workshops', '2015 Seminar on Reinsurance',
    '2016 CAS Annual Meeting', '2016 CAS Interactive Live Streaming: CLRS', '2016 CAS Interactive Live Streaming: Spring',
    '2016 CAS Seminar on Reinsurance', '2016 CAS Spring Meeting', '2016 CLRS & Workshops',
    '2016 Ratemaking and Product Management Seminar & Workshops', '2017 CAS Annual Meeting',
    '2017 CAS Interactive Live Stream: Casualty Loss Reserve Seminar', '2017 CAS Interactive Live Streaming: Annual',
    '2017 CAS Interactive Live Streaming: Spring', '2017 CAS Seminar on Reinsurance', '2017 CAS Spring Meeting',
    '2017 CLRS & Workshops', '2017 Ratemaking and Product Management Seminar & Workshops', '2018 CAS Annual Meeting',
    '2018 CAS Interactive Live Stream: Casualty Loss Reserve Seminar', '2018 CAS Interactive Live Streaming: Annual',
    '2018 CAS Interactive Live Streaming: Spring', '2018 CAS Reinsurance Seminar', '2018 CAS Spring Meeting',
    '2018 CLRS & Workshops', '2018 RPM Seminar & Workshops', '2019 CAS Annual Meeting',
    '2019 CAS Interactive Live Stream: Casualty Loss Reserve Seminar', '2019 CAS Interactive Live Streaming: Annual',
    '2019 CAS Interactive Live Streaming: RPM', '2019 CAS Interactive Live Streaming: Reinsurance Seminar',
    '2019 CAS Interactive Live Streaming: Spring', '2019 CAS Reinsurance Seminar', '2019 CAS Spring Meeting',
    '2019 CLRS & Workshops', '2019 RPM Seminar & Workshops', '2020 Virtual CAS Annual Meeting',
    '2020 Virtual CAS Ratemaking, Product, and Modeling Seminar', '2020 Virtual CAS Seminar on Reinsurance',
    '2020 Virtual CAS Spring Meeting', '2020 Virtual CLRS', '2021 CAS Annual Meeting',
    '2021 CAS Interactive Live Streaming: Annual', '2021 Virtual CAS Ratemaking, Product, and Modeling Seminar',
    '2021 Virtual CAS Seminar on Reinsurance', '2021 Virtual CAS Spring Meeting', '2021 Virtual CLRS',
    '2022 CAS Annual Meeting', '2022 CAS Interactive Live Streaming: Annual', '2022 CAS Interactive Live Streaming: CLRS',
    '2022 CAS Interactive Live Streaming: Spring', '2022 CAS Spring Meeting', '2022 CLRS & Workshops',
    '2022 RPM Virtual Seminar', '2022 Virtual CAS Seminar on Reinsurance', '2023 CAS Annual Meeting',
    '2023 CAS Interactive Live Streaming: Annual', '2023 CAS Interactive Live Streaming: CLRS',
    '2023 CAS Interactive Live Streaming: Spring', '2023 CAS Interactive Livestream RPM',
    '2023 CAS Interactive Livestream Reinsurance', '2023 CAS Seminar on Reinsurance', '2023 CAS Spring Meeting',
    '2023 CLRS & Workshops', '2023 Ratemaking, Product, and Modeling Seminar', '2024 CAS Annual Meeting',
    '2024 CAS Interactive Live Streaming: Annual', '2024 CAS Interactive Live Streaming: CLRS',
    '2024 CAS Interactive Live Streaming: Spring', '2024 CAS Interactive Livestream RPM',
    '2024 CAS Interactive Livestream Reinsurance', '2024 CAS Ratemaking, Product, and Modeling Seminar',
    '2024 CAS Seminar on Reinsurance', '2024 CAS Spring Meeting', '2024 CLRS & Workshops'
]

event_code = [
    '15Annual', '15LIVECLRS', '15Spring', '15CLRS', '2015RPM', '2015REINS', '16Annual', '16LIVECLRS', '16LIVESPR',
    '2016REINS', '16Spring', '16CLRS', '2016RPM', '17Annual', '17CLRSLS', '17LIVEANN', 'LIVESPR', '2017REINS',
    '17Spring', '17CLRS', '17RPM', '18Annual', '18CLRSLS', '18LIVEANN', 'LIVESPR', '2018REINS', '18Spring', '18CLRS',
    '18RPM', '19Annual', '19CLRSLS', '19LIVEANN', '19LIVERPM', 'REILS', 'LIVESPR', '2019REINS', '19Spring', '19CLRS',
    '19RPM', '20VirAnn', '20VRPM', '20VREINS', '20VirSprng', '20VRCLRS', '21Annual', '21LiveAnn', '21VRPM',
    '21VREINS', '21VirSprng', '21VRCLRS', '22Annual', '22LiveANN', '22LiveCLRS', '22LiveSpr', '22Spring', '22CLRS',
    'RPMLive22', '22VREINS', '23Annual', '23Annual Livestream', '23CLRS Livestream', '23LiveSpr', '23LIVERPM',
    '23LIVEREI', '23REI', '23Spring', '23CLRS', '23RPM', '24Annual', '24Annual Livestream', '24CLRS Livestream',
    '24Spring Livestream', '24RPM Livestream', '24Reins Livestream', '24RPM', '24Reins', '24Spring', '24CLRS'
]

parent_event = [
    '15Annual', '15CLRS', '15Spring', '15CLRS', '2015RPM', '2015REINS', '16Annual', '16CLRS', '16Spring',
    '2016REINS', '16Spring', '16CLRS', '2016RPM', '17Annual', '17CLRS', '17Annual', '17Spring', '2017REINS',
    '17Spring', '17CLRS', '17RPM', '18Annual', '18CLRS', '18Annual', '18Spring', '2018REINS', '18Spring',
    '18CLRS', '18RPM', '19Annual', '19CLRS', '19Annual', '19RPM', '2019REINS', '19Spring', '2019REINS', '19Spring',
    '19CLRS', '19RPM', '20VirAnn', '20VRPM', '20VREINS', '20VirSprng', '20VRCLRS', '21Annual', '21Annual',
    '21VRPM', '21VREINS', '21VirSprng', '21VRCLRS', '22Annual', '22Annual', '22CLRS', '22Spring', '22Spring',
    '22CLRS', 'RPMLive22', '22VREINS', '23Annual', '23Annual', '23CLRS', '23Spring', '23RPM', '23REI', '23REI',
    '23Spring', '23CLRS', '23RPM', '24Annual', '24Annual', '24CLRS', '24Spring', '24RPM', '24Reins', '24RPM',
    '24Reins', '24Spring', '24CLRS'
]

parent_event_map = pl.DataFrame({
    'Event_Title': event_title,
    'Event_Code': event_code,
    'parent_event': parent_event
})


In [ ]:
df = primary_df.with_columns(
    home_key = pl.concat_str(
        [pl.col("City"), pl.col("State"), pl.col("Country")],
        separator=", ", ignore_nulls=True
    ),
    meet_key = pl.concat_str(
        [pl.col("Location_City"), pl.col("Location_State")],
        separator=", ", ignore_nulls=True
    ),
)

df = (
    df
    .join(
        cache_df.rename({"lat":"home_lat","lon":"home_lon"}),
        left_on="home_key", right_on="place_key", how="left"
    )
    .join(
        cache_df.rename({"lat":"mtg_lat","lon":"mtg_lon"}),
        left_on="meet_key", right_on="place_key", how="left"
    )
)

In [ ]:
import math

R_KM = 6371.0088
MI_PER_KM = 0.621371

def haversine_km(s: dict) -> float | None:
    lat1, lon1, lat2, lon2 = s["home_lat"], s["home_lon"], s["mtg_lat"], s["mtg_lon"]
    if None in (lat1, lon1, lat2, lon2):
        return None
    φ1, λ1, φ2, λ2 = map(math.radians, (lat1, lon1, lat2, lon2))
    dφ = φ2 - φ1
    dλ = λ2 - λ1
    a = math.sin(dφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R_KM * c

df = (
    df
    .with_columns(
        pl.struct(["home_lat","home_lon","mtg_lat","mtg_lon"])
          .map_elements(haversine_km)
          .alias("distance_km")
    )
    .with_columns(
        (pl.col("distance_km") * MI_PER_KM).alias("distance_mi")
    )
)

df = df.join(parent_event_map,on=['Event_Title','Event_Code'],how='inner')